In [1]:
# 导入相关包
import re as re
import numpy as np
import math as mt
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# 提取能量集中度txt文件中指定视场数据
# 参数：path -> 能量集中度txt文件保存位置
#       field_list -> 指定需要提取数据的视场，默认提取所有视场
# 返回：字典类型 -> 视场和其对应能量集中度函数曲线
def extract_data(path, field_list=-1):
    f = open(path, encoding='utf-16')
    txtData = f.readlines()
    
    params = {}
    count = 0
    index =0
    isFindField = False
    
    for i in range(len(txtData)):
        find_field = re.findall(r"Field", txtData[i])
        if find_field:
            fieldData = []
            index = i
            count += 1
            field = "Field" + str(count)
            isFindField = True

        if (i >= index+4) and isFindField:
            find_data = re.findall(r"\d+\.?\d*", txtData[i])
            if find_data:
                fieldData.append(find_data)
            else:
                params[field] = np.array(fieldData, dtype=np.float64)

        if i == len(txtData) - 1:
            params[field] = np.array(fieldData, dtype=np.float64)

    # 将指定视场能量集中度数据提取出来
    paramsSub = {}
    if field_list == -1:
        return params
    else:
        for i in field_list:
            paramsSub[('Field'+str(i))] = params[('Field'+str(i))]
        
        return paramsSub

In [12]:
# 将能量集中度数据转换为单个点的能量
# 参数： energy_list -> 不同半径list对应的能量
#        X_half_range -> X方向取点的半长度
#        Y_half_range -> Y方向取点的半长度
#        step ->X和Y方向取点步长，默认0.05
# 返回： pointEnergy -> 各个坐标点的能量值
def energy_in_point(energy_list, X_half_range, Y_half_range, step=0.05):
    X_range = np.arange(-X_half_range, X_half_range, step)
    Y_range = np.arange(-Y_half_range, Y_half_range, step)
    radiusStep = energy_list[0,1] - energy_list[0,0]
    count = np.zeros(energy_list.shape[1], dtype=np.int32)
    # 计算每个环内坐标点的个数
    for i in X_range:
        for j in Y_range:
            r = mt.sqrt(mt.pow(i,2)+mt.pow(j,2))
            index = int(r/radiusStep)
            if index < count.shape[0]:
                count[index] += 1

    #计算每个坐标点的能量
    pointEnergy = np.zeros((X_range.shape[0], Y_range.shape[0]),dtype = np.float64)
    for i in range(X_range.shape[0]):
        for j in range(Y_range.shape[0]):
            r = mt.sqrt(mt.pow(X_range[i],2)+mt.pow(Y_range[j],2))
            index = int(r/radiusStep) + 1
            if index < count.shape[0]:
                pointEnergy[i,j] = (energy_list[1,index] - energy_list[1,index-1])/count[index-1]
                if (energy_list[1,index] - energy_list[1,index-1])<0:
                    print("xiaoyu0")
    
    return pointEnergy

In [9]:
# 收集每个像元能量数据
# 参数： energy_list -> 单个坐标点的能量数组
#        X_pixel -> X方向像素
#        Y_pixel -> Y方向像素
# 返回： 每个像素能量数据
def energy_in_pixel(energy_list, X_pixel, Y_pixel):
    pixelEnergy = np.zeros((X_pixel, Y_pixel), dtype=np.float64)
    X_step = energy_list.shape[1] / X_pixel
    Y_step = energy_list.shape[0] / Y_pixel
    for i in range(Y_pixel):
        for j in range(X_pixel):
            pixelEnergy[i, j] = np.sum(energy_list[int(i*X_step):int((i+1)*X_step), int(j*Y_step):int((j+1)*Y_step)])
    
    return pixelEnergy

In [10]:
# 扩展面光源成像，每个像元能量数据
# 参数： energy_list -> 单个坐标点的能量数组
#        X_shiftPoint -> X方向移动坐标点个数
#        Y_shiftPoint -> Y方向移动坐标点个数
#        X_step -> X方向移动步长，以单个坐标点为单位
#        Y_step -> Y方向移动步长，以单个坐标点为单位
# 返回： 每个像素能量数据
def energy_in_pixel_with_planesource(energy_list, X_shiftPoint, Y_shiftPoint, X_step, Y_step):
    X_range = np.arange(-X_shiftPoint/2, X_shiftPoint/2, X_step, dtype=np.int8)
    Y_range = np.arange(-Y_shiftPoint/2, Y_shiftPoint/2, Y_step, dtype=np.int8)
    energyTemX = energy_list
    energyTemY = energy_list
    energyTemSum = np.zeros_like(energy_list)
    planeEnergy = np.zeros_like(energy_list)
    count = 0
    for i in X_range:
        energyTemX = np.roll(energy_list, i, axis=1)
        if i > 0:
            energyTemX[:, 0:i] = 0
        else:
            energyTemX[:, (energy_list.shape[1]+i):energy_list.shape[1]] = 0
        
        for j in Y_range:
            count += 1
            energyTemY = np.roll(energyTemX, j, axis=0)
            if j > 0:
                energyTemY[0:j, :] = 0
            else:
                energyTemY[(energy_list.shape[0]+j):energy_list.shape[0], :] = 0
            
            energyTemSum += energyTemY
    
    planeEnergy = energyTemSum / count
    
    return planeEnergy

In [93]:
# 将np数组写入Excel表格中
# 参数： data_list -> 需要写入的np数组
#        sheet_title -> Excel sheet名称
#        file_name -> Excel名称
#        startcol -> 数据写入起始列
#        startrow -> 数据写入起始行
# 返回： 无
def data_save_excel(data_list, sheet_title, file_name, startcol, startrow):
    data = pd.DataFrame(data_list)
    writer = pd.ExcelWriter(file_name)
    data.to_excel(writer, sheet_title, startcol=startcol, startrow=startrow)
    writer.save()
    writer.close()

In [95]:
# 定义需要计算像元能量集中度的视场序号
fieldNums = [1, 5]
# 定义需要计算能量集中度的文件名称
fieldFiles = np.array(['data/wave4.txt', 'data/wave5.txt'])
dictKey = np.array(['wave4','wave5'])

energyDict = {}
countI = 0
countJ = 0
# 计算各个文件中需要计算的像元能量集中度
for i in fieldFiles:
    countI += 1
    for j in fieldNums:
        countJ += 1
        #提取能量集中度txt文件中指定视场数据
        energyData = extract_data(i, fieldNums)
        # 将能量集中度数据转换为单个点的能量
        pointEnergy = energy_in_point(energyData['Field'+str(j)].T, 37.5, 37.5)
        #扩展面光源成像，每个像元能量数据
        planeEnergy = energy_in_pixel_with_planesource(pointEnergy, 300, 300, 10, 10)
        # 收集每个像元能量数据------点光源
        pixelEnergy = energy_in_pixel(pointEnergy, 5, 5)
        # 将np数组字典中
        data_save_excel(pixelEnergy, 'sheet_title', dictKey[countI-1]+'-Field'+str(j)+'.xlsx', 0, 0)
        # 收集每个像元能量数据------点光源
        pixelEnergy = energy_in_pixel(planeEnergy, 5, 5)
        # 将np数组字典中
        data_save_excel(pixelEnergy, 'sheet_title', dictKey[countI-1]+'-Field'+str(j)+'.xlsx', 0, 0)


